In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2
scikit-learn-intelex==20230228.214242


In [2]:
!pip install scikit-learn==1.2.2 

In [1]:
import os
import uuid
import pickle

import pandas as pd

import mlflow

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [2]:
year = 2022
month = 3
taxi_type = 'yellow'

input_file = f'../../../data/yellow/yellow_tripdata_2022-03.parquet'
output_file = f'output/{taxi_type}/{year:04d}-{month:02d}.parquet'

In [3]:
input_file

'../../../data/yellow/yellow_tripdata_2022-03.parquet'

In [4]:
def generate_uuids(n):
    ride_ids = []
    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))
    return ride_ids

def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    df['ride_id'] = generate_uuids(len(df))

    return df

def prepare_dictionaries(df: pd.DataFrame):
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [5]:
def load_model():
    with open('model.bin', 'rb') as f_in:
        dv,model = pickle.load(f_in)
    return (dv,model)

In [8]:
def apply_model(input_file, output_file):

    df = read_dataframe(input_file)
    dicts = prepare_dictionaries(df)
    
    dv,model = load_model()
    X_val = dv.transform(dicts)
    y_pred = model.predict(X_val)

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['tpep_pickup_datetime'] = df['tpep_pickup_datetime']
    df_result['PULocationID'] = df['PULocationID']
    df_result['DOLocationID'] = df['DOLocationID']
    df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    #df_result['model_version'] = run_id
    
   
    #df_result.to_parquet(output_file, index=False)
    print(y_pred.std())

In [9]:
apply_model(input_file=input_file, output_file=output_file)

1.9184653865522705e-13


In [13]:
df = pd.read_parquet(input_file)

In [14]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')